In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN1011"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 6    #number of convolution to have in the network
pooling_stride = [1, 1, 1, 1, 1, 32]
weights_shape = [[7, n_variables, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32]]
bias_shape = [32, 32, 32, 32, 32, 32]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [7]:
print pos_weight

3.18601822124


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [10]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [11]:
model.build_conv_net(pos_weight = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.286201, fmeasure_max= 0.385928, precision= 0.239947, recall= 0.985481
Train Random guess:  , auc= 0.225312fmeasure_max= 0.355204, precision= 0.219068, recall= 0.938294
Validation Iter 0, auc= 0.212014, fmeasure_max= 0.371101, precision= 0.227823, recall= 1.000000
Validation Random guess:  auc= 0.224122, fmeasure_max= 0.373490, precision= 0.230645, recall= 0.981132
Minibatch Loss= 1.027637
Tarin Iter 100, auc= 0.472346, fmeasure_max= 0.487746, precision= 0.329712, recall= 0.936728
Train Random guess:  , auc= 0.252137fmeasure_max= 0.404959, precision= 0.255004, recall= 0.983025
Validation Iter 100, auc= 0.422421, fmeasure_max= 0.427432, precision= 0.278442, recall= 0.919383
Validation Random guess:  auc= 0.230272, fmeasure_max= 0.377205, precision= 0.235095, recall= 0.953688
Minibatch Loss= 1.041106
Tarin Iter 200, auc= 0.614558, fmeasure_max= 0.555450, precision= 0.587174, recall= 0.526978
Train Random guess:  , auc= 0.235062fmeasure_max= 0.358625, precision= 0.2211

In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../models/model" + exp_name + "/model")

In [14]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.393246 F1_max = 0.427069 prec = 0.293756 rec = 0.781926
Test Random guess: auc= 0.234843 fmeasure_max= 0.375790 , precision= 0.231422 , recall= 0.998986


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})